### Analyze MovieLens Dataset using Apache Spark

I have loaded the **MovieLens** dataset (25M) in a bucket in the Object Storage
and shows here how to query it using **Spark SQL** in a Notebook Session

conda env used: **pyspark32_p38_cpu_v1**

Dataset is here: https://grouplens.org/datasets/movielens/

In [1]:
import logging
import warnings
import os
import ads
from os import path
from pyspark.sql import SparkSession
import re

warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.ERROR)

URL FORMAT is oci://{BUCKET}@{NAMESPACE}/{REF_NAME}

In [2]:
ads.set_auth(auth='resource_principal') 

In [3]:
%load_ext sparksql_magic
%config SparkSql.max_num_rows=100

In [4]:
# create a spark session
spark = SparkSession \
    .builder \
    .appName("Python Spark MovieLens") \
    .config("spark.driver.cores", str(1)) \
    .config("spark.executor.cores", str(8)) \
    .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")

In [5]:
movies = spark.read.parquet("oci://WORKSHOP@frqap2zhtzbe/movies.parquet").cache()
      
# the dataframe as a sql view so we can perform SQL on it
movies.createOrReplaceTempView("MOVIES")

In [6]:
ratings = spark.read.parquet("oci://WORKSHOP@frqap2zhtzbe/ratings.parquet").cache()

# the dataframe as a sql view so we can perform SQL on it
ratings.createOrReplaceTempView("RATINGS")

In [7]:
%%sparksql

DESC MOVIES

col_name,data_type,comment
movieId,string,null
title,string,null
genres,string,null


In [8]:
%%sparksql

DESC RATINGS

col_name,data_type,comment
userId,string,null
movieId,string,null
rating,string,null
timestamp,string,null


In [9]:
%%sparksql

SELECT * FROM MOVIES LIMIT 10

movieId,title,genres
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,Jumanji (1995),Adventure|Children|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama|Romance
5,Father of the Bride Part II (1995),Comedy
6,Heat (1995),Action|Crime|Thriller
7,Sabrina (1995),Comedy|Romance
8,Tom and Huck (1995),Adventure|Children
9,Sudden Death (1995),Action
10,GoldenEye (1995),Action|Adventure|Thriller


In [12]:
%%sparksql

SELECT * FROM RATINGS LIMIT 10

userId,movieId,rating,timestamp
1,296,5.0,1147880044
1,306,3.5,1147868817
1,307,5.0,1147868828
1,665,5.0,1147878820
1,899,3.5,1147868510
1,1088,4.0,1147868495
1,1175,3.5,1147868826
1,1217,3.5,1147878326
1,1237,5.0,1147868839
1,1250,4.0,1147868414


#### Film con il più alto numero di ratings

In [13]:
%%sparksql

SELECT title, genres, ROUND(AVG(rating), 1) as avg_rating, count(*) as num_ratings 
FROM RATINGS, MOVIES WHERE MOVIES.movieId = RATINGS.movieId 
GROUP BY title, genres
HAVING count(*) > 10
ORDER BY num_ratings DESC LIMIT 10

title,genres,avg_rating,num_ratings
Forrest Gump (1994),Comedy|Drama|Romance|War,4.0,81491
"Shawshank Redemption, The (1994)",Crime|Drama,4.4,81482
Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,4.2,79672
"Silence of the Lambs, The (1991)",Crime|Horror|Thriller,4.2,74127
"Matrix, The (1999)",Action|Sci-Fi|Thriller,4.2,72674
Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Sci-Fi,4.1,68717
Jurassic Park (1993),Action|Adventure|Sci-Fi|Thriller,3.7,64144
Schindler's List (1993),Drama|War,4.2,60411
Braveheart (1995),Action|Drama|War,4.0,59184
Fight Club (1999),Action|Crime|Drama|Thriller,4.2,58773


In [14]:
%%sparksql

SELECT title, ROUND(AVG(rating), 1) as avg_rating, count(*) as num_ratings 
FROM RATINGS, MOVIES WHERE MOVIES.movieId = RATINGS.movieId 
and genres = "Adventure" 
GROUP BY title
HAVING count(*) > 10
ORDER BY avg_rating DESC LIMIT 10

title,avg_rating,num_ratings
Belle and Sebastien (Belle et Sébastien) (2013),3.8,21
The Spy Who Loved Flowers (1966),3.7,118
Mountains of the Moon (1990),3.7,170
Wolf Totem (2015),3.7,18
Neon Bull (2015),3.7,13
King Jack (2015),3.6,32
Billy Budd (1962),3.6,32
"Mark of Zorro, The (1940)",3.6,897
Time Traveller: The Girl Who Leapt Through Time (2010),3.5,22
"Wind and the Lion, The (1975)",3.5,195


#### Test the API

In [15]:
 query_result_df = spark.sql("""
    SELECT MOVIES.title, ROUND(AVG(RATINGS.rating), 1) as avg_rating, count(*) as num_ratings 
    FROM RATINGS, MOVIES 
    WHERE MOVIES.movieId = RATINGS.movieId 
    and MOVIES.genres = "Adventure" 
    GROUP BY MOVIES.title
    HAVING count(*) > 10
    ORDER BY avg_rating DESC LIMIT 10
    """)

In [16]:
print(query_result_df.toJSON().collect())

['{"title":"Belle and Sebastien (Belle et Sébastien) (2013)","avg_rating":3.8,"num_ratings":21}', '{"title":"Mountains of the Moon (1990)","avg_rating":3.7,"num_ratings":170}', '{"title":"The Spy Who Loved Flowers (1966)","avg_rating":3.7,"num_ratings":118}', '{"title":"Wolf Totem (2015)","avg_rating":3.7,"num_ratings":18}', '{"title":"Neon Bull (2015)","avg_rating":3.7,"num_ratings":13}', '{"title":"Mark of Zorro, The (1940)","avg_rating":3.6,"num_ratings":897}', '{"title":"King Jack (2015)","avg_rating":3.6,"num_ratings":32}', '{"title":"Billy Budd (1962)","avg_rating":3.6,"num_ratings":32}', '{"title":"Time Traveller: The Girl Who Leapt Through Time (2010)","avg_rating":3.5,"num_ratings":22}', '{"title":"Wind and the Lion, The (1975)","avg_rating":3.5,"num_ratings":195}']


In [17]:
query_result_df.show()

+--------------------+----------+-----------+
|               title|avg_rating|num_ratings|
+--------------------+----------+-----------+
|Belle and Sebasti...|       3.8|         21|
|Mountains of the ...|       3.7|        170|
|The Spy Who Loved...|       3.7|        118|
|   Wolf Totem (2015)|       3.7|         18|
|    Neon Bull (2015)|       3.7|         13|
|Mark of Zorro, Th...|       3.6|        897|
|    King Jack (2015)|       3.6|         32|
|   Billy Budd (1962)|       3.6|         32|
|Time Traveller: T...|       3.5|         22|
|Wind and the Lion...|       3.5|        195|
+--------------------+----------+-----------+



#### Usate per salvare in format Parquet

Le istruzioni usate per salvare i dati, letti da file csv, in formato parquet sull'Object Storage

In [ ]:
# movies.write.parquet("oci://WORKSHOP@frqap2zhtzbe/movies.parquet")

In [ ]:
# ratings.write.parquet("oci://WORKSHOP@frqap2zhtzbe/ratings.parquet")